In [16]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver  
import requests
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
from googlesearch import search

base = 'https://www.amazon.com'
thirdWheel = '/s?k='

In [8]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
categ = []
for box in boxes:
    # actual = box.get('href')
    # link = base + actual
    # print(link)
    # links.append(link)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print(link)
    links.append(link)
    categ.append(som)
    print(som)


[<a aria-label="Headsets" class="a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window" href="/s/?_encoding=UTF8&amp;k=gaming%20headsets&amp;ref=nb_sb_noss_2&amp;pd_rd_w=6ACBB&amp;content-id=amzn1.sym.12129333-2117-4490-9c17-6d31baf0582a&amp;pf_rd_p=12129333-2117-4490-9c17-6d31baf0582a&amp;pf_rd_r=ZA2SVGG3ZXKMQ3WVCJY0&amp;pd_rd_wg=nkfIx&amp;pd_rd_r=bb00d5c0-64e8-4557-9039-905872eadedc&amp;ref_=pd_hp_d_atf_unk"><div class="a-section a-spacing-none _fluid-quad-image-label-v2_style_fluidImageContainer__2SOMr"><img alt="Headsets" class="_fluid-quad-image-label-v2_style_fluidLandscapeImage__2euAK _fluid-quad-image-label-v2_style_fluidImage__gTBjA" data-a-hires="https://images-na.ssl-images-amazon.com/images/G/01/AmazonExports/Fuji/2021/June/Fuji_Quad_Headset_2x._SY232_CB667159060_.jpg" src="https://images-na.ssl-images-amazon.com/images/G/01/AmazonExports/Fuji/2021/June/Fuji_Quad_Headset_1x._SY116_CB667159060_.jpg"/></div><div class="a-section a-spacing-no

In [6]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [4]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [17]:
products = []
proxies = get_valid_proxies()
total_samples = 0
for link in links:
    url = link
    while url:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", url)
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        # driver.get(url)
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # time.sleep(10)
        # html = driver.page_source
        # driver.quit()
        headers = rotate_user_agent(proxy)
        response = req.get(url, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(url, headers = {"User-Agent": UserAgent().random})
        if response.status_code == 200:
            soup = bsp(response.text, 'lxml')
        # soup = bsp(html, 'lxml')
            print(soup)
            productes = soup.find_all('a', class_ = 'a-url-normal s-faceout-url aok-block a-text-normal')
            print(productes)
            for product in productes:
                product = url + product.get('href')
                products.append(product)
                print(product)
                total_samples += 1
            
            nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
            if nextP:
                url = base + nextP.get('href')

VISITING :  https://www.amazon.com/s?k=Headsets
VISITING :  https://www.amazon.com/s?k=Headsets
VISITING :  https://www.amazon.com/s?k=Headsets
VISITING :  https://www.amazon.com/s?k=Headsets


ConnectionError: HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /s?k=Headsets (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017D31187990>: Failed to resolve 'www.amazon.com' ([Errno 11001] getaddrinfo failed)"))